In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import random

from sklearn.metrics import classification_report, roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [ ]:
triggers_actions = pd.read_csv(r'C:\Users\User\triggers\before_logreg.csv')

In [ ]:
triggers_actions.head()

In [ ]:
# делим данные на train и test, на одних и тех же данных предскажем отдельно веротяность факта взаимодействия и вероятность успеха взаимодействия 

X = triggers_actions[['trigger', 'time_since_last_action', 'triggers_since_last_action', 'last_successful_type']]
y_interaction = triggers_actions['action_date']  # таргет для взаимодействия

# Разделение данных на train/test с фиксацией индексов
X_train, X_test, y_train_interaction, y_test_interaction = train_test_split(X, y_interaction, test_size=0.3, random_state=42)

In [ ]:
# Сохраняем индексы, которые попали в train/test
train_indices = X_train.index
test_indices = X_test.index


# Шаг 2: Используем те же индексы для задачи предсказания успеха
y_success = triggers_actions['result']  # таргет для успеха


In [ ]:

# Создаем выборки для успеха, используя сохраненные индексы
X_train_success = X.loc[train_indices]
X_test_success = X.loc[test_indices]

y_train_success = y_success.loc[train_indices]
y_test_success = y_success.loc[test_indices]


In [ ]:
### модели 
# модель для взаимодействия 
logreg_interaction = LogisticRegression(class_weight='balanced')


In [ ]:
logreg_interaction.fit(X_train, y_train_interaction)


In [ ]:
interaction_probs = logreg_interaction.predict_proba(X_test)[:, 1]


In [ ]:

# модель для успеха 
logreg_success = LogisticRegression(class_weight='balanced', multi_class='ovr')


In [ ]:
logreg_success.fit(X_train_success, y_train_success)


In [ ]:
importance_df.head(15)

In [ ]:
# получаем коэффициенты модели
feature_importance = logreg_success.coef_[0]  # для мультиклассовой модели берем первый массив

# создаем DataFrame для наглядности
feature_names = X_train_success.columns  # названия признаков из обучающего набора
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# сортируем признаки по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# визуализация
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Feature Importance in Logistic Regression')
plt.gca().invert_yaxis()  # чтобы важные признаки были сверху
plt.show()

In [ ]:
success_probs = logreg_success.predict_proba(X_test_success)[:, 1]


In [ ]:
# добавляем обе вероятности в датафрейм для анализа
triggers_actions.loc[test_indices, 'interaction_prob'] = interaction_probs


In [ ]:
triggers_actions.loc[test_indices, 'success_prob'] = success_probs

In [ ]:
triggers_actions.head(3)

In [ ]:
triggers_actions.dropna(inplace = True)

In [ ]:
triggers_actions.to_csv(r'C:\Users\User\triggers\after_logreg.csv',index = False)